# 00 - Prerequisites: Azure Storage and SFTP Setup

**IMPORTANT:** This setup is now automated via shell scripts that run on your local machine.

## Prerequisites

Before running Databricks notebooks, you need to set up Azure infrastructure on your **local machine**:

1. **Azure CLI** installed and configured
2. **SSH** utilities installed
3. **Databricks CLI** installed and configured

## Setup Steps (Local Machine)

Run these scripts on your local machine:

### 1. Azure Infrastructure Setup
```bash
cd scripts
./setup_azure_infrastructure.sh
```

This script will:
- Generate SSH key pair
- Create Azure resource group
- Create source and target storage accounts with SFTP enabled
- Create containers
- Configure SFTP users
- Upload sample CSV files
- Display connection details

### 2. Databricks Secrets Setup
```bash
cd scripts
./setup_databricks_secrets.sh
```

This script will:
- Create Databricks secret scope
- Store SFTP connection details in secrets
- Upload SSH private key to DBFS

## Next Steps

After running the setup scripts, proceed to:
- **Notebook 01**: `01_infrastructure_setup.ipynb` - Install SFTP package in Databricks
- **Notebook 02**: `02_uc_connection_setup.ipynb` - Configure Unity Catalog connections
- **Notebook 03**: `03_dlt_pipeline.ipynb` - Run the DLT pipeline